In [ ]:
#Install necessary dependencies
!pip install -U langchain-community
!pip install langchain langchain_google_genai google-generativeai google-search-results yfinance textblob --q

#Import required modules
import yfinance as yf
from langchain.agents import initialize_agent, AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import Tool
from langchain.utilities import SerpAPIWrapper
from textblob import TextBlob
from google.colab import userdata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.18
    Uninstalling langchain-0.3.18:
      Successfully uninstalled langchain-0.3.18
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 367.0 kB/s eta 0:00:00


In [ ]:
#Retrieve API keys
GOOGLE_API_KEY = userdata.get("google_api_key")
SERPAPI_KEY = userdata.get("serpai_api_key")

if not GOOGLE_API_KEY:
    raise ValueError("Google API key not found. Make sure to add it to Colab secrets.")

if not SERPAPI_KEY:
    raise ValueError("SerpAPI key not found. Make sure to add it to Colab secrets.")

llm = ChatGoogleGenerativeAI(
    google_api_key=GOOGLE_API_KEY,
    model="gemini-2.0-flash-exp",
    temperature=0
)

#Google News Search Agent
search_tool = SerpAPIWrapper(serpapi_api_key=SERPAPI_KEY)

news_tool = Tool(
    name="GoogleNewsSearch",
    func=search_tool.run,
    description="Fetches top news articles from Google using SerpAPI."
)

#Financial Data Agent
def get_stock_data(ticker="AAPL"):
    stock = yf.Ticker(ticker)
    data = stock.history(period="5d")
    return {
        "latest_price": stock.info["currentPrice"],
        "open": stock.info["open"],
        "high": stock.info["dayHigh"],
        "low": stock.info["dayLow"],
        "market_cap": stock.info["marketCap"],
    }

financial_tool = Tool(
    name="StockDataFetcher",
    func=lambda x: get_stock_data(x),
    description="Fetches real-time stock data for a given company ticker."
)

#Sentiment Analysis Agent
def analyze_sentiment(news_text):
    analysis = TextBlob(news_text)
    return "Positive" if analysis.sentiment.polarity > 0 else "Negative" if analysis.sentiment.polarity < 0 else "Neutral"

sentiment_tool = Tool(
    name="SentimentAnalyzer",
    func=lambda x: analyze_sentiment(x),
    description="Analyzes sentiment of news articles or text related to a stock."
)

#Investment Advice Agent
def investment_advice(stock="AAPL"):
    data = get_stock_data(stock)
    if data["latest_price"] > data["open"]:
        return "The stock is trending upward. Consider reviewing market conditions before investing."
    else:
        return "The stock is experiencing a decline. Monitor for further movement."

investment_tool = Tool(
    name="InvestmentAdvisor",
    func=lambda x: investment_advice(x),
    description="Provides general investment insights based on stock trends."
)

#Initialize the agent with multiple tools
tools = [news_tool, financial_tool, sentiment_tool, investment_tool]

agent_chain = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=10
)

<ipython-input-2-c32dde25f299>:73: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_chain = initialize_agent(


In [ ]:
#Invoke the agents
response = agent_chain.invoke("Summarize today's latest news on Apple stock and analyze sentiment. Provide investment insights.")

print(response)



> Entering new AgentExecutor chain...
Okay, I need to get the latest news on Apple stock, analyze the sentiment of the news, and then provide investment insights based on the news and sentiment.

Action: GoogleNewsSearch
Action Input: Apple stock
Observation: {'type': 'finance_results', 'title': 'Apple Inc', 'exchange': 'NASDAQ', 'stock': ' AAPL', 'currency': 'USD', 'price': 247.1, 'previous_close': 245.83}
Thought:I need to get the latest news on Apple stock, analyze the sentiment of the news, and then provide investment insights based on the news and sentiment. I should use GoogleNewsSearch to get the latest news articles.

Action: GoogleNewsSearch
Action Input: Apple stock news
Observation: [{'title': "If I Could Only Buy 1 Warren Buffett Artificial Intelligence (AI) Stock, This Would Be It (Hint: It's Not Apple)", 'link': 'https://www.fool.com/investing/2025/02/21/if-i-could-only-buy-1-warren-buffett-ai-stock/', 'source': 'The Motley Fool', 'date': '2 hours ago', 'thumbnail': 'ht